In [ ]:
def compute_loss(model, inputs, targets, criterion, weights = None, compute_grad=False):
    '''
    Computes gradients of model with given inputs and targets and loss function.
    Optionally backpropagates to compute gradients for weights.
    Procedure depends on whether we have one model for each source or not
    :param model: Model to train with
    :param inputs: Input mixture
    :param targets: Target sources
    :param criterion: Loss function to use (L1, L2, ..) это имеется ввиду Loss1, Loss2 или L1-norm, L2-norm? 
    :param compute_grad: Whether to compute gradients
    :return: Model outputs, Average loss over batch

    я так понимаю, что возвращается
    '''
    all_outputs = {}

    if weights is None:
        weights = [1.0] * len(criterion)

    loss_values = np.zeros_like(criterion)

    if model.separate:
        avg_loss = 0.0
        num_sources = 0
        for inst in model.instruments:
            output = model(inputs, inst)
            loss = criterion(output[inst], targets[inst])

            if compute_grad:
                loss.backward()

            avg_loss += loss.item()
            num_sources += 1

            all_outputs[inst] = output[inst].detach().clone()

        avg_loss /= float(num_sources)
    else:
        loss = 0
        all_outputs = model(inputs)
        for inst in all_outputs.keys():
            loss += criterion(all_outputs[inst], targets[inst])

        if compute_grad:
            loss.backward()

        avg_loss = loss.item() / float(len(all_outputs))

    return all_outputs, avg_loss